In [1]:
from model import *
from data import *

Using TensorFlow backend.


In [2]:
from importlib import reload  # Python 3.4+ only.
import data
import model as model_functions

reload(data)
reload(model_functions)

<module 'model' from 'D:\\drive\\MyDriveBackup\\9.Semester\\Masterarbeit\\masterarbeit - git\\Käferbäume\\Unet detect trees\\model.py'>

In [3]:
WIDTH = 256#128 #256
HEIGHT = 256#128 #256

In [5]:
model = unet(input_size=(WIDTH,HEIGHT,3), nr_classes=len(COLOR_DICT))
# model.load_weights('../../../Models/detect-tree/multi classes 128 - 256/4th-run-weights-improvement-20-0.0544.h5')
model.load_weights('../../../Models/detect-tree/5th-run-128-256-weights-improvement-18-0.0959.h5') # new

# load_model('../../../Models/detect-tree/multi classes 128 - 256/4th-run-weights-improvement-20-0.0544.h5', custom_objects={'softMaxAxis1': softMaxAxis1, 'mean_io_u_13': tf.keras.metrics.MeanIoU})#, 'mean_io_u_13': tf.keras.metrics.MeanIoU, 'MeanIoU': tf.keras.metrics.MeanIoU})

In [6]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_25[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_26[0][0]                  
____________________________________________________________________________________________

# process all images in a folder + subfolders:

In [7]:
# folder_path = "../../../Data/Borkenkaefer/data"
# save_masks_path = "../../../Data/Borkenkaefer/data_trunc_detected_masks"
# save_cropped_path = "../../../Data/Borkenkaefer/data_trunc_detected_cropped"
# save_multiplied_path = "../../../Data/Borkenkaefer/data_trunc_detected_multiplied"

folder_path = "../../../Data/BFW/filtered"
save_masks_path = "../../../Data/BFW/data_trunc_detected_masks"
save_cropped_path = "../../../Data/BFW/data_trunc_detected_cropped"
save_multiplied_path = "../../../Data/BFW/data_trunc_detected_multiplied"

In [8]:
def create_mask(pred_mask):
    pred_mask = np.argmax(pred_mask, axis=-1)
#     pred_mask = pred_mask[..., np.newaxis]
    color_label_matrix = data.get_color_label_matrix(pred_mask[0])
    return color_label_matrix

In [14]:
from scipy.ndimage import interpolation

def crop_image(file_name, mask):
    # this method crops left and right from the image such that only the trunk is left
    full_size_image = image.load_img(file_name)
    
    # resize the mask to match the original image size using interpolation
    img_width = full_size_image.width
    img_height = full_size_image.height
    mask = interpolation.zoom(mask, [img_height/mask.shape[0], img_width/mask.shape[1]])
    
    # keep columns, which have an avg of more than x%
    full_size_image = image.img_to_array(full_size_image)
    full_size_image /= 255
#     x = 0.2
    x = 0.4
    avg_cols = np.mean(mask, axis=0)
    more_than_x = avg_cols > x
    cropped = full_size_image[:, more_than_x]
    
#     plt.imshow(full_size_image)
#     plt.show()
    
#     plt.imshow(cropped)
#     plt.show()
    return np.array(cropped*255, dtype=np.uint8)
    

In [10]:
def multiply_image_mask(file_name, mask):
    # this method multiplies the mask with the image, which results in a background of zeros
    org_image = image.load_img(file_name)
    
    # resize the mask to match the original image size using interpolation
    img_width = org_image.width
    img_height = org_image.height
    mask = interpolation.zoom(mask, [img_height/mask.shape[0], img_width/mask.shape[1]])
    
    org_image = image.img_to_array(org_image)
    org_image /= 255
    
    discrete_mask = mask > 0.5 # set the mask to 0s and 1s
    
    return np.array(discrete_mask[:,:,np.newaxis] * org_image * 255, dtype=np.uint8) # newaxis: broadcast mask over RGB dimensions

In [11]:
def set_image_background_zero(file_name, mask):
    # this method sets all discovered background pixels to zero by using the method from "crop images", which only handles whole columns;
    # this counteracts artefacts, which are produced by the "detect Trunk" network i.e. sometimes it predicts zero in the middle of the trunk...
    # this method makes sure that nothing important is set to zero
    org_image = image.load_img(file_name)
    
    # resize the mask to match the original image size using interpolation
    img_width = org_image.width
    img_height = org_image.height
    mask = interpolation.zoom(mask, [img_height/mask.shape[0], img_width/mask.shape[1]])
    
    # set columns to zero, which have an avg of less than x%
    org_image = image.img_to_array(org_image)
    org_image /= 255
    x = 0.5
    avg_cols = np.mean(mask, axis=0)
    less_than_x = avg_cols < x
    org_image[:, less_than_x] = 0
        
    return np.array(org_image*255, dtype=np.uint8)

In [16]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np


for subdir, dirs, files in os.walk(folder_path):
    new_subdir = subdir.replace(folder_path, save_masks_path)
    new_cropped_subdir = subdir.replace(folder_path, save_cropped_path)
    new_multiplied_subdir = subdir.replace(folder_path, save_multiplied_path)
    
    for file in files:
        file_name = os.path.join(subdir, file)
        
        # load and preprocess image
        sample_image = image.load_img(file_name, target_size=(HEIGHT, WIDTH))
        sample_image = image.img_to_array(sample_image)
        sample_image /= 255
        
        # predict mask and transform it to an image
        pred = model.predict(sample_image[tf.newaxis, ...])
        
        #-------------------------------
        # uncomment this, if you want to save the masks too
        # save predicted mask to a new directory
        mask = np.array(create_mask(pred), dtype=np.uint8)
        img = Image.fromarray(mask, 'RGB')
        if not os.path.exists(new_subdir):
            os.makedirs(new_subdir)
        img.save(file_name.replace(folder_path, save_masks_path))
        
        #-------------------------------
        # crop images according to their mask 
        cropped = crop_image(file_name, pred[0,:,:,0])
        
        # save the cropped versions of the images to a new directory
        cropped_img = Image.fromarray(cropped, 'RGB')
        if not os.path.exists(new_cropped_subdir):
            os.makedirs(new_cropped_subdir)
        cropped_img.save(file_name.replace(folder_path, save_cropped_path))
        
        #-------------------------------
#         # multiply image and mask
# #         mult_image = multiply_image_mask(file_name, pred[0,:,:,0])
#         # set background to zero; use this method to only set the images left and right columns to zero
#         mult_image = set_image_background_zero(file_name, pred[0,:,:,0])
#         # save the new image to a new directory
#         mult_image = Image.fromarray(mult_image, 'RGB')
#         if not os.path.exists(new_multiplied_subdir):
#             os.makedirs(new_multiplied_subdir)
#         mult_image.save(file_name.replace(folder_path, save_multiplied_path))
